# Movie Classification using Naive Bayes Classifier

## 1. Data
The dataset is composed of 3 files.

* **movies.dat** - movie information (title and genre)
* **users.dat** - user information (including gender, age, occupation...)
* **ratings.dat** - movie ratings given by users

In [1]:
import pandas as pd
import numpy as np

from IPython.display import display, HTML

In [2]:
DATA_PATH = "../data"

data_movies = pd.read_csv(DATA_PATH + "/movies.dat", delimiter="::", header=None, engine="python",
                         names=["MovieID", "Title", "Genres"])

data_users = pd.read_csv(DATA_PATH + "/users.dat", delimiter="::", header=None, engine="python",
                         names=["UserID", "Gender", "Age", "Occupation", "Zip-code"])

data_ratings = pd.read_csv(DATA_PATH + "/ratings.dat", delimiter="::", header=None, engine="python",
                         names=["UserID", "MovieID", "Rating", "Timestamp"])


Some simple data visualization...

In [3]:
print ("Movies dataset")
display(data_movies.head())
display(data_movies.info())

Movies dataset


,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3883 entries, 0 to 3882
Data columns (total 3 columns):
MovieID    3883 non-null int64
Title      3883 non-null object
Genres     3883 non-null object
dtypes: int64(1), object(2)
memory usage: 91.1+ KB


None

In [4]:
print ("Users dataset")
display(data_users.head())
display(data_users.info())

Users dataset


,UserID,Gender,Age,Occupation,Zip-code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6040 entries, 0 to 6039
Data columns (total 5 columns):
UserID        6040 non-null int64
Gender        6040 non-null object
Age           6040 non-null int64
Occupation    6040 non-null int64
Zip-code      6040 non-null object
dtypes: int64(3), object(2)
memory usage: 236.0+ KB


None

In [5]:
print ("Ratings dataset")
display(data_ratings.head())
display(data_ratings.info())

Ratings dataset


,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000209 entries, 0 to 1000208
Data columns (total 4 columns):
UserID       1000209 non-null int64
MovieID      1000209 non-null int64
Rating       1000209 non-null int64
Timestamp    1000209 non-null int64
dtypes: int64(4)
memory usage: 30.5 MB


None

In [6]:
#Data cleaning

#Joining df
data = pd.merge(data_ratings, data_users, how="inner", on="UserID")
data = pd.merge(data, data_movies, how="inner", on="MovieID")


In [7]:
#Feature Extraction

#Droping unwanted columns
#data = data.drop("Zip-code", 1)
#data = data.drop("Timestamp", 1)
data = data.drop("Title", 1)
#data = data.drop("MovieID", 1)
#data = data.drop("UserID", 1)

data['Genres'].apply(lambda x: x.split('|'))
data['Gender'] = data['Gender'].apply(lambda x: 0 if x == 'M' else 1)
display(data.head())


,UserID,MovieID,Rating,Timestamp,Gender,Age,Occupation,Zip-code,Genres
0,1,1193,5,978300760,1,1,10,48067,Drama
1,2,1193,5,978298413,0,56,16,70072,Drama
2,12,1193,4,978220179,0,25,12,32793,Drama
3,15,1193,4,978199279,0,25,7,22903,Drama
4,17,1193,5,978158471,0,50,1,95350,Drama


In [8]:

#data['Genres'].apply(lambda x: x.split('|'))
#genres_set = {}
#for i in data['Genres']:
#    genres = i.split('|')
#    for g in genres:
#        if g not in genres_set:
#            genres_set[g] = 0
#        genres_set[g] += 1
#       
#print (genres_set)
#len(genres_set)


In [9]:
data.drop("Genres", 1)
data = data.drop("MovieID", 1)
data = data.drop("UserID", 1)


from sklearn.naive_bayes import GaussianNB

